In [61]:
%load_ext autoreload
%autoreload 2
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *
from training_utils import *
from prokbert_tokenizer import ProkBERTTokenizer
from prok_datasets import *
from ProkBERTDataCollator import *
    
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

train_model_output_dir = ''
train_model_name = 'LCA_repr_k6s1_l2048hs1536_h16_l16_mlm02_rp8_rndp1_rkv'

model_params = {'model_outputpath': '/scratch/fastscratch/NBL/trained_models/test/mini0',
                'model_name' : 'mini0'}
dataset_params = {'dataset_path' : '../data/preprocessed/pretraining.h5'}
pretraining_params = {}


prokbert_config = ProkBERTConfig()
_ = prokbert_config.get_and_set_model_parameters(model_params)
_ = prokbert_config.get_and_set_dataset_parameters(dataset_params)
_ = prokbert_config.get_and_set_pretraining_parameters(pretraining_params)


# Kell egy dataset
hdf_file = '../data/preprocessed/pretraining.h5'

ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=10000, max_iteration_over_ds=5)

X = ds[0:3][:,0:10]
#torch.stack(ds[0:3])


2023-08-14 19:49:16,448 - INFO - Dataset size: 482


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PRETRAIN_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/pretraining.yaml
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert


In [62]:
tokenizer = get_training_tokenizer(prokbert_config)
prokbert_dc = get_data_collator_for_overlapping_sequences(tokenizer, prokbert_config)



SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
Loading the datacollator class!
Collator Parameters:
  Number of tokens masked to left:  3
  Number of tokens masked to right: 2
  Probability of restoring a masked token: 0.8
  Probability of changing to a random token: 0.01
  MLM Probability: 0.05
  Default token type: torch.int16



In [63]:
prokbert_config.model_params

[m_exists, cp_dir, cp, cps] = check_model_existance_and_checkpoint(prokbert_config.model_params['model_outputpath'], 
                                     prokbert_config.model_params['model_name'])
# Conclusion: The current model extsts and can be resumed. 

# Cheking initiation model, if training required



{'model_name': 'mini0',
 'model_outputpath': '/scratch/fastscratch/NBL/trained_models/test/mini0',
 'vocab_size': 4101,
 'hidden_size': 384,
 'num_hidden_layers': 6,
 'num_attention_heads': 6,
 'max_position_embeddings': 1024,
 'intermediate_size': 2048,
 'position_embedding_type': 'relative_key_query',
 'ResumeTraining': True,
 'resume_or_initiation_model_path': ''}

model_path: /scratch/fastscratch/NBL/trained_models/test/mini0
/scratch/fastscratch/NBL/trained_models/test/mini0
The 0 is the largest checkpoint!


In [66]:
# checking hdf dataset
hdf_file_exists, ds_size = check_hdf_dataset_file(prokbert_config)





2023-08-14 19:49:30,559 - INFO - Dataset size: 482


Checking whether the file exists or not!
Loading and creating a IterableProkBERTPretrainingDataset


In [55]:
prokbert_config.dataset_params


{'dataset_path': '',
 'pretraining_dataset_data': [[]],
 'dataset_class': 'IterableProkBERTPretrainingDataset',
 'input_batch_size': 10000,
 'dataset_iteration_batch_offset': 0,
 'max_iteration_over_dataset': 10}

In [18]:
print('Loading the datacollator class!')
prokbert_dc = ProkBERTDataCollator(tokenizer,
                                  mask_to_left=prokbert_config.data_collator_params['mask_to_left'], 
                                  mask_to_right=prokbert_config.data_collator_params['mask_to_right'],
                                  mlm_probability =   prokbert_config.data_collator_params['mlm_probability'],
                                  replace_prob =prokbert_config.data_collator_params['replace_prob'],
                                  random_prob = prokbert_config.data_collator_params['random_prob'])
print(str(prokbert_dc))

inputs, labels = prokbert_dc.torch_mask_tokens(ds[0:3][:,0:30])



Loading the datacollator class!
Collator Parameters:
  Number of tokens masked to left:  3
  Number of tokens masked to right: 2
  Probability of restoring a masked token: 0.8
  Probability of changing to a random token: 0.01
  MLM Probability: 0.05
  Default token type: torch.int16



### Pretraining folyamata

Vannak előkészítő feladatok:
  * tokenizáló loadolása
  * collator osztály loadolása
  * Meglévő modellek ellenőrzése, checkpoint-ok ellenőrzése
  * dataset betöltése és inicializálása, ellenőrzése (megfelel-e a követelményeknek vagy sem)


In [9]:
tokenizer = get_training_tokenizer(prokbert_config)
prokbert_dc = get_data_collator_for_overlapping_sequences(tokenizer, prokbert_config)

#inputs, labels = prokbert_dc.torch_mask_tokens(sequences)
# Kell egy paraméter szet a computation-nek, tokenizer-nek, stb. 

# Paraméterszett a datacollator-nak
# Paraméterszett a modell-nek
# Paraméterszett a training-nek. 
# IsResumeTraining és ilyenek


#prokbert_config.computation_params

SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
Loading the datacollator class!
Collator Parameters:
  Number of tokens masked to left:  3
  Number of tokens masked to right: 2
  Probability of restoring a masked token: 0.8
  Probability of changing to a random token: 0.01
  MLM Probability: 0.05
  Default token type: torch.int16



In [11]:
prokbert_config.default_torchtype

torch.int16